In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [2]:
path = '../raw_data/train'

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=90,
    validation_split = 0.2)

In [4]:
valid_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split = 0.2)

In [5]:
train_generator = train_datagen.flow_from_directory(
    path,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='training',
    seed = 123)

Found 3460 images belonging to 16 classes.


In [6]:
val_generator = valid_datagen.flow_from_directory(
    path, # same directory as training data
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    seed = 123)

Found 857 images belonging to 16 classes.


In [8]:
# class_weight = {0:4. ,
#                 1:2., 
#                 2:1.5 ,
#                 3:2.,
#                 4:3.,
#                 5:2.,
#                 6:1.5,
#                 7:4.,
#                 8:2.,
#                 9:3.,
#                 10:1.,
#                 11:4.,
#                 12:3.,
#                 13:2.,
#                 14:3.,
#                 15:2.
#                }

In [9]:
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(256,256,3))

model=Sequential()
for layers in vgg16_model.layers:
    model.add(layers)
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dense(4096,activation='relu'))
for layer in model.layers:
    layer.trainable=False
model.add(Dense(16,activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 64, 64, 256)      

In [ ]:
# # Set the first layers to be untrainable
# model.trainable = False
# # Add layers to the mdoel
# flatten_layer = layers.Flatten()
# dense_layer = layers.Dense(100, activation='relu')
# prediction_layer = layers.Dense(16, activation='softmax')

# model = Sequential([model,
#                     flatten_layer,
#                     dense_layer,
#                     prediction_layer])
# model.summary()

In [24]:
# model.compile(optimizer=optimizers.SGD(learning_rate = 0.1),
#           loss=SparseCategoricalCrossentropy(from_logits=True),
#           metrics=['accuracy'])

In [10]:
optimizer=RMSprop(lr=0.001,rho=0.9,epsilon=1e-08,decay=0.0)

In [11]:
model.compile(optimizer,loss=SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [12]:
learning_rate_reduction=ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.5,minlr=0.00001)

In [22]:
# callbacks = EarlyStopping(
#     monitor='val_loss',
#     patience=3,
#     mode='min')

In [14]:
history=model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // 32,
    validation_data = val_generator,
    validation_steps = val_generator.samples // 32,
    epochs = 15,
    callbacks = [learning_rate_reduction])

Epoch 1/15
108/108 [==============================] - 669s 6s/step - loss: 2.6477 - accuracy: 0.1604 - val_loss: 2.5852 - val_accuracy: 0.1839
Epoch 2/15
108/108 [==============================] - 679s 6s/step - loss: 2.5355 - accuracy: 0.2908 - val_loss: 2.4919 - val_accuracy: 0.3810
Epoch 3/15
108/108 [==============================] - 841s 8s/step - loss: 2.4711 - accuracy: 0.3696 - val_loss: 2.4452 - val_accuracy: 0.3846
Epoch 4/15
108/108 [==============================] - 820s 8s/step - loss: 2.4308 - accuracy: 0.3929 - val_loss: 2.4087 - val_accuracy: 0.4387
Epoch 5/15
108/108 [==============================] - 836s 8s/step - loss: 2.3996 - accuracy: 0.4414 - val_loss: 2.3873 - val_accuracy: 0.4736
Epoch 6/15
108/108 [==============================] - 2925s 27s/step - loss: 2.3763 - accuracy: 0.4522 - val_loss: 2.3669 - val_accuracy: 0.4688
Epoch 7/15
108/108 [==============================] - 3187s 30s/step - loss: 2.3580 - accuracy: 0.4883 - val_loss: 2.3471 - val_accuracy: 0.

In [15]:
model.save(r'../model/roxane_vgg_3')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../model/roxane_vgg_3/assets
